In [1]:
import cv2
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import json
import random
import os
import re
import unicodedata
from tqdm.auto import tqdm
import uuid
from imagedegrade import np as degrade
import math

In [2]:
def filter_label(label, len_range, max_space):
    if label is None:
        return False
    if len(label) < len_range[0] or len(label) > len_range[1]:
        return False
    if label.count(' ') > max_space:
        return False
    if label.isascii() == False:
        return False    
    for char in "\n\r\xad\xa0":
        if char in label:
            return False
    return True

def get_labels_from_json(
    n_key_map={
        "date": 3_00000,
        "designation": 60_00000,
        "variety": 5_00000,
        "region": 2_00000,
        "province": 5_00000,
        "country": 1_00000,
        "winery": 9_0000,
    },
    len_range=(0, 100),
    max_space=100
):
    wine_data = json.load(open("../datasets/vin_db.json", encoding="utf-8"))
    key_list = ["title", "designation", "variety", "region_1", "province", "country", "winery"]
    wine_data = [{k:d[k] for k in key_list} for d in wine_data]
    for wine in wine_data: 
        wine["title"] = re.sub("[^0-9]", "", wine["title"]) # Remove non alpha numeric characters, we only want the dates
        wine["date"] = wine.pop("title") # Rename key
        wine["region"] = wine.pop("region_1") # Rename key
    labels = {}
    for key in wine_data[0].keys():
        labels[key] = list(filter(lambda label: filter_label(label, len_range, max_space), [wine[key] for wine in wine_data] )) # Remove all odd values
        # labels[key] = list(map(lambda x: unicodedata.normalize("NFKD", re.sub(r'[^\x00-\x7F]', '', x).replace("\r\n", '').replace('\xad', '')), labels[key])) # Remove all \r and \n
        random.shuffle(labels[key]) # Shuffle
    for i in range(len(labels["designation"])): # split by 3 spaces and add remaning data at the end
        splitted_designtation = labels["designation"][i].split(' ')
        part1 = ' '.join(splitted_designtation[:3])
        part2 = ' '.join(splitted_designtation[3:])
        labels["designation"][i] = part1
        if type(part2) == str and len(part2) > 0:
            labels["designation"].append(part2)
    random.shuffle(labels["designation"]) # Shuffle again
    res_labels = []
    for key in n_key_map.keys(): # Cut to wanted length
        res_labels = res_labels + labels[key][:n_key_map[key]]
    return res_labels #remove duplicate values

def get_labels_from_wine_names( len_range=(0, 100), max_space=100):
    labels = open("../datasets/wine-names.txt").readlines()
    return list(filter(lambda label: filter_label(label, len_range, max_space) , labels))
    

labels = get_labels_from_json(len_range=(3, 32), max_space=3)
labels = labels + get_labels_from_wine_names(len_range=(3, 32), max_space=3)
labels = list(set(labels))
for i in range(1500, 2050): # add dates
    labels.append(str(i))
random.shuffle(labels) #shuffle
labels = labels

In [3]:
def degrade_image(image_array, blur_range, jpeg_range):
    image = degrade.blur(image_array, random.uniform(blur_range[0], blur_range[1])).astype(np.uint8)
    # image = degrade.noise(image, random.randint(1, 2)).astype(np.uint8)
    # image = degrade.saltpepper(image, random.uniform(0.05, 0.08)).astype(np.uint8)
    image = degrade.jpeg(image, random.randint(jpeg_range[0], jpeg_range[1])).astype(np.uint8)
    return image.astype(np.uint8)

def apply_wave_filter(img, wave_range):
    img_output = np.zeros(img.shape, dtype=img.dtype)
    mean_color = img[0][0]
    rows, cols, _ = img.shape
    sigma = random.uniform(wave_range[0], wave_range[1])
    # Wave effect
    for i in range(rows):
        for j in range(cols):
            offset_y = 0
            offset_y = -int(sigma * math.sin(2 * 3.14 * j / cols))
            if i+offset_y < rows:
                img_output[i,j] = img[(i+offset_y)%rows,j]
            else:
                img_output[i,j] = mean_color
    return img_output

def get_rand_rgb():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def generate_images(labels, directory, configs):
    # font_files = os.listdir("../datasets/fonts/")
   
    if os.path.exists(directory) == False: os.makedirs(directory)    
    data_file = open(os.path.join(directory, "data.txt"), "a+")
    labels = labels
    for font_file, config in tqdm(configs.items()):
        for label in tqdm(labels):
            try:
                font_color = get_rand_rgb()
                bg_color =  get_rand_rgb()
                while (abs(sum(font_color) - sum(bg_color)) <= config["min_color_diff"]):
                    font_color = get_rand_rgb()
                    bg_color =  get_rand_rgb()
                file_name = f"{uuid.uuid1()}.png"
                font_size = random.randint(config["font_size_range"][0], config["font_size_range"][1])
                image_size = (
                    int(len(label) * config["image_width_x"]),
                    random.randint(config["image_height_range"][0], config["image_height_range"][1]),

                )
                image = Image.new(
                    'RGB',
                    image_size,
                    bg_color
                )
                draw = ImageDraw.Draw(image)
                font = ImageFont.truetype("../datasets/fonts/" + font_file, font_size)
                draw.text(
                    (
                        int(image_size[0] * random.uniform(config["pos_x_range_x"][0], config["pos_x_range_x"][1])),
                        int(image_size[1] * random.uniform(config["pos_y_range_x"][0], config["pos_y_range_x"][1])),
                    ),
                    str(label),
                    font=font,
                    fill=font_color
                )
                image.save(os.path.join(directory, file_name))
                image = degrade_image(np.asarray(image), config["blur_range"], config["jpeg_range"])
                image = apply_wave_filter(image, config["wave_range"])
                image = Image.fromarray(image)
                image = image.resize((256, 64))
                image.save(os.path.join(directory, file_name))
                data_file.write(f"{file_name} {label}\n")            
            except Exception as e :
                None
    data_file.close()

# Store configurations for each font
configs = {
    "Times Sans Serif.ttf": {
        "font_size_range": (60, 75),
        "image_width_x": 40,
        "image_height_range": (125, 135),
        "pos_x_range_x": (0.05, 0.1),
        "pos_y_range_x": (0.1, 0.35),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (90, 98),
        "wave_range": (-10, 10),
        "min_color_diff": 80,
    },
    "SCRIPTIN.ttf": {
        "font_size_range": (100, 110),
        "image_width_x": 75,
        "image_height_range": (300, 350),
        "pos_x_range_x": (0.15, 0.18),
        "pos_y_range_x": (0.025, 0.05),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 250,
    },
    "Handwritten.ttf": {
        "font_size_range": (100, 110),
        "image_width_x": 48,
        "image_height_range": (180, 200),
        "pos_x_range_x": (0.1, 0.1),
        "pos_y_range_x": (0.1, 0.3),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "Handwritten - Italic.ttf": {
        "font_size_range": (100, 110),
        "image_width_x": 45,
        "image_height_range": (180, 200),
        "pos_x_range_x": (0.05, 0.1),
        "pos_y_range_x": (0.1, 0.3),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "GreatVibes.ttf": {
        "font_size_range": (100, 110),
        "image_width_x": 60,
        "image_height_range": (180, 200),
        "pos_x_range_x": (0.015, 0.1),
        "pos_y_range_x": (0.1, 0.3),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "Creattion.otf": {
        "font_size_range": (110, 120),
        "image_width_x": 60,
        "image_height_range": (150, 170),
        "pos_x_range_x": (0.015, 0.1),
        "pos_y_range_x": (0.02, 0.1),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "CMTiempo.ttf": {
        "font_size_range": (50, 60),
        "image_width_x": 40,
        "image_height_range": (150, 170),
        "pos_x_range_x": (0.1, 0.2),
        "pos_y_range_x": (0.1, 0.3),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 120,
    },
    "CASLBLSI.TTF": {
        "font_size_range": (50, 60),
        "image_width_x": 50,
        "image_height_range": (150, 170),
        "pos_x_range_x": (0.1, 0.2),
        "pos_y_range_x": (0.1, 0.3),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "Bickham.otf": {
        "font_size_range": (150, 160),
        "image_width_x": 80,
        "image_height_range": (250, 300),
        "pos_x_range_x": (0.1, 0.2),
        "pos_y_range_x": (0.1, 0.3),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "Bernhard.otf": {
        "font_size_range": (150, 160),
        "image_width_x": 80,
        "image_height_range": (250, 300),
        "pos_x_range_x": (0.1, 0.2),
        "pos_y_range_x": (0.2, 0.4),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-20, 20),
        "min_color_diff": 200,
    },
    "Bemboza.ttf": {
        "font_size_range": (120, 160),
        "image_width_x": 60,
        "image_height_range": (250, 300),
        "pos_x_range_x": (0.1, 0.3),
        "pos_y_range_x": (0.2, 0.4),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (80, 100),
        "wave_range": (-12, 12),
        "min_color_diff": 100,
    },
    "AlexBrush.ttf": {
        "font_size_range": (80, 90),
        "image_width_x": 50,
        "image_height_range": (220, 270),
        "pos_x_range_x": (0.1, 0.3),
        "pos_y_range_x": (0.2, 0.4),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (100, 100),
        "wave_range": (-5, 5),
        "min_color_diff": 200,
    },
    "Akzidenz.ttf": {
        "font_size_range": (60, 75),
        "image_width_x": 65,
        "image_height_range": (125, 135),
        "pos_x_range_x": (0.1, 0.45),
        "pos_y_range_x": (0.2, 0.5),
        "blur_range": (0.5, 1.5),
        "jpeg_range": (90, 98),
        "wave_range": (-10, 10),
        "min_color_diff": 80,
    },
}

generate_images(labels, '../datasets/multi-fonts-generated-text/', configs)



  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]